In [ ]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib


path_submissions = '/'

target_name = 'target'
scores_folds = {}
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)

import os
import glob
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
def count_unique(series):
    return len(np.unique(series))
def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], 
                                                          np.arange(0,600)], 
                                                         names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

def grab_more_f(trade_path, book_path, stock_id):
    
    trade_train = pd.read_parquet(trade_path)
    book_train = pd.read_parquet(book_path)
    
    merge = trade_train.merge(book_train,on=['time_id','seconds_in_bucket'],how='right')
    merge = merge.sort_values(['time_id','seconds_in_bucket'])
    merge.reset_index(inplace=True,drop=True)
    merge['new_feature'] = merge['bid_price1']-merge['ask_price1']
    merge['new_feature'] = abs(merge['new_feature'])
    merge['new_feature'] = merge['new_feature']/merge['price']
    
    
    merge['stock_id']=stock_id
    merge['row_id']=merge['stock_id'].astype(str)+'-'+merge['time_id'].astype(str)
    dict_1 = merge.groupby(['row_id'])['new_feature'].mean().to_dict()
    dict_2 = {}#merge.groupby(['row_id'])['new_feature2'].mean().to_dict()
    dict_3 = {}#merge.groupby(['row_id'])['new_feature3'].mean().to_dict()
    dict_4 = {}#merge.groupby(['row_id'])['new_feature4'].mean().to_dict()

    #dict_8 = merge.groupby(['row_id'])['order_count_isna'].max().to_dict()
    #dict_7 = merge.groupby(['row_id'])['bid_price1_isna'].sum().to_dict()
    
    merge = merge[merge['seconds_in_bucket']>300]
    dict_7 = {}#merge.groupby(['row_id'])['new_feature'].mean().to_dict()
    dict_5 = {}#merge.groupby(['row_id'])['new_feature2'].mean().to_dict()
    dict_6 = {}#merge.groupby(['row_id'])['new_feature3'].mean().to_dict()
    dict_8 = {}#merge.groupby(['row_id'])['new_feature4'].mean().to_dict()
    #print(len(merge.time_id.unique()))
    return pd.DataFrame(),pd.DataFrame(),dict_1,dict_2,dict_3,dict_4,dict_5,dict_6,dict_7,dict_8
from scipy.stats import kurtosis
from scipy.stats import skew
#kurtosis(x).

def preprocessor_book(file_path):
    
    df = pd.read_parquet(file_path)
    #df = ffill(df)
    #calculate return etc
    df['wap'] = calc_wap(df)
    df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
    
    df['wap2'] = calc_wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    #df['wap_balance'] = abs(df['wap'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    #df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    #df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    #df['wap_dif']=df['wap'] - df['wap2']
    #df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['log_return_diff']=abs(df['log_return'] - df['log_return2'])
    
    #df['h_spread_l1'] = df['ask_price1'] - df['bid_price1']
    #df['h_spread_l2'] = df['ask_price2'] - df['bid_price2']
    #df['v_spread_b'] = df['bid_price1'] - df['bid_price2']
    #df['v_spread_a'] = df['ask_price1'] - df['ask_price2']
    

    #dict for aggregate
    create_feature_dict = {
        #'ask_price1':[count_unique],
        #'ask_price2':[count_unique],
        #'bid_price1':[count_unique],
        #'bid_price2':[count_unique],
        'log_return':[realized_volatility,np.std],
        'log_return2':[realized_volatility,np.std],
        #'wap_balance':[np.mean,np.max,np.min,np.std, kurtosis, skew],
        'price_spread':[np.mean],#,np.max,np.min,np.std, kurtosis, skew],
        #'bid_spread':[np.mean,np.max,np.min,np.std, kurtosis, skew],
        'ask_spread':[np.max],
        #'volume_imbalance':[np.mean,np.max,np.min,np.std, kurtosis, skew],
        'total_volume':[np.min],
        'wap':[np.mean],
        #'wap2':[np.mean,np.max,np.min,np.std, kurtosis, skew],
        #'wap_dif':[np.mean,np.std,np.max,np.min,kurtosis,skew],
        'log_return_diff':[np.mean],#,np.std,np.max,np.min,kurtosis,skew],
        #"bid_ask_spread":[np.mean]
        
        
        #'h_spread_l1':[np.mean],
        #'h_spread_l2':[np.mean],
        #'v_spread_b':[np.mean],
        #'v_spread_a':[np.mean],
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
      
        

        
        
    ######groupby / last XX seconds
    last_seconds = [300, 150, 50]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
                
    
    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)        
        
        
        
    df['stock_id']=stock_id
    df['row_id']=df['stock_id'].astype(str)+'-'+df['time_id'].astype(str)
    
  
    return df_feature

def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)

    aggregate_dictionary = {
        'log_return':[realized_volatility],#,np.max,np.min,np.std, kurtosis, skew],
        'seconds_in_bucket':[count_unique, sum],#,np.max,np.min,np.std, kurtosis, skew],
        'size':[np.sum],#,np.max,np.min,np.std, kurtosis, skew],
       # 'order_count':[np.mean,np.max,np.min,np.std, kurtosis, skew, np.sum],
        #'my_trade_feature_1':[np.mean,np.max,np.min,np.std, kurtosis, skew],
        'price':[np.std]#, np.max, np.min, kurtosis, skew, count_unique]#,np.std,np.max,np.min
    }
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    ######groupby / last XX seconds
    last_seconds = [300, 150, 50]
    
    for second in last_seconds:
        second = 600 - second
    
        df_feature_sec = df.query(f'seconds_in_bucket >= {second}').groupby('time_id').agg(aggregate_dictionary)
        df_feature_sec = df_feature_sec.reset_index()
        
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))
        
        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
            
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)  
    
    
    df['stock_id']=stock_id
    df['row_id']=df['stock_id'].astype(str)+'-'+df['time_id'].astype(str)
    
    return df_feature

def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')

        
        df_tmp2,df_tmp3,dict_1,dict_2,dict_3,dict_4,dict_5,dict_6,dict_7,dict_8= grab_more_f(file_path_trade, file_path_book, stock_id)

        df_tmp['new_feat']=[dict_1.get(n,np.nan) for n in df_tmp['row_id']]
        
        return pd.concat([df,df_tmp])       
        
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

#kurtosis median quantile
def add_features_by_time(df, feature, by='time_id'):
    temp = df.groupby(by)[feature].mean().to_dict()
    df['avg1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]

    temp = df.groupby(by)[feature].std().to_dict()
    df['std1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]

    temp = df.groupby(by)[feature].max().to_dict()
    df['max1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]

    temp = df.groupby(by)[feature].min().to_dict()
    df['min1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]
    
    temp = df.groupby(by)[feature].skew().to_dict()
    df['skew1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]
    
    temp = df.groupby(by)[feature].median().to_dict()
    df['median1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]    
    
    temp = df.groupby(by)[feature].quantile(.25).to_dict()
    df['quantile_025_1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]  

    temp = df.groupby(by)[feature].quantile(.75).to_dict()
    df['quantile_075_1_'+feature+'_by_'+by] = [temp[n] for n in df[by]]  
    
    temp = df.groupby(by)[feature].apply(pd.DataFrame.kurt).to_dict()
    df['kurt1_'+feature+'_by_'+by] = [temp[n] for n in df[by]] 
    
    #.apply(pd.DataFrame.kurt)
    
    return df






test = pd.read_csv(data_dir + 'test.csv')
test_ids = test.stock_id.unique()
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)
df_test = test.merge(df_test, on = ['row_id'], how = 'left')
df_test.head()


train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train['row_id'] = train['stock_id'].astype(str)+'-'+train['time_id'].astype(str)
train_ids = train.stock_id.unique()
df_train = preprocessor(list_stock_ids= train_ids, is_train = True)
df_train.drop(['stock_id','time_id'],axis=1,inplace=True,errors='ignore')
df_train = train.merge(df_train, on = ['row_id'], how = 'left')

cols = [

        'log_return_realized_volatility',
        'log_return2_realized_volatility_450',
        'log_return2_realized_volatility_300',
    
        'trade_log_return_realized_volatility_300',
        'trade_seconds_in_bucket_count_unique',
        'trade_seconds_in_bucket_count_unique_450',
        'trade_size_sum',
        'trade_size_sum_300',
        'trade_size_sum_450',
        
       ]

from tqdm import tqdm
for col in tqdm(cols):
    df_train = add_features_by_time(df_train, col, by='time_id')
    df_test = add_features_by_time(df_test, col, by='time_id')

df_test['ff1']=df_test['log_return_realized_volatility_300']/df_test['log_return_realized_volatility']
df_train['ff1']=df_train['log_return_realized_volatility_300']/df_train['log_return_realized_volatility']


df_train['size_tau'] = np.sqrt( 1/ df_train['trade_seconds_in_bucket_count_unique'] )
df_test['size_tau'] = np.sqrt( 1/ df_test['trade_seconds_in_bucket_count_unique'] )

df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)
import os
import sys
import numpy as np
import pandas as pd
import random
from collections import Counter, defaultdict
from sklearn import model_selection
import lightgbm as lgbm
# ---- GroupKFold ----
class GroupKFold(object):
    """
    GroupKFold with random shuffle with a sklearn-like structure
    """

    def __init__(self, n_splits=4, shuffle=True, random_state=42):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, group=None):
        return self.n_splits

    def split(self, X, y, group):
        kf = model_selection.KFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state=self.random_state)
        unique_ids = group.unique()
        for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_ids)):
            # split group
            tr_group, va_group = unique_ids[tr_group_idx], unique_ids[va_group_idx]
            train_idx = np.where(group.isin(tr_group))[0]
            val_idx = np.where(group.isin(va_group))[0]
            yield train_idx, val_idx
            

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 500,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
    'verbose': -1
  }

cols = '''0.2955 ± 0.0117	log_return_realized_volatility
0.1010 ± 0.0011	log_return_realized_volatility_300
0.0439 ± 0.0012	stock_id
0.0396 ± 0.0010	log_return2_realized_volatility
0.0267 ± 0.0001	log_return2_realized_volatility_300
0.0071 ± 0.0001	log_return2_realized_volatility_450
0.0070 ± 0.0002	log_return_realized_volatility_450
0.0047 ± 0.0002	trade_log_return_realized_volatility
0.0067 ± 0.0003	trade_log_return_realized_volatility_450
0.0035 ± 0.0001	trade_log_return_realized_volatility_300
0.0113 ± 0.0003	min1_log_return2_realized_volatility_300_by_time_id
0.0101 ± 0.0007	quantile_025_1_log_return2_realized_volatility_450_by_time_id
0.0077 ± 0.0002	price_spread_mean_550
0.0057 ± 0.0001	max1_trade_seconds_in_bucket_count_unique_450_by_time_id
0.0055 ± 0.0001	new_feat
0.0052 ± 0.0002	quantile_025_1_trade_size_sum_450_by_time_id
0.0043 ± 0.0001	max1_trade_seconds_in_bucket_count_unique_by_time_id
0.0031 ± 0.0000	log_return_realized_volatility_550
0.0025 ± 0.0006	avg1_trade_size_sum_300_by_time_id
0.0024 ± 0.0001	log_return2_realized_volatility_550
0.0024 ± 0.0001	trade_price_std_300
0.0023 ± 0.0002	std1_log_return_realized_volatility_by_time_id
0.0020 ± 0.0001	trade_price_std_550
0.0017 ± 0.0002	median1_trade_seconds_in_bucket_count_unique_by_time_id
0.0016 ± 0.0002	kurt1_trade_size_sum_by_time_id
0.0008 ± 0.0003	ff1
0.0007 ± 0.0001	min1_trade_log_return_realized_volatility_300_by_time_id
0.0007 ± 0.0001	total_volume_amin_450
0.0007 ± 0.0001	quantile_075_1_trade_size_sum_by_time_id
0.0007 ± 0.0001	median1_log_return2_realized_volatility_300_by_time_id
0.0007 ± 0.0001	wap_mean_550
0.0007 ± 0.0001	ask_spread_amax_450
0.0007 ± 0.0001	log_return2_std_550
0.0007 ± 0.0001	log_return_diff_mean_300
0.0007 ± 0.0001	bid_spread_amin_300
0.0007 ± 0.0001	size_tau'''


cols = [n.split('\t')[1] for n in cols.split('\n')]

models = []                          # models
scores = 0.0                         # validation score

groups = df_train['time_id']
ncols = cols#+[n]
X_copy = df_train[[n for n in df_train.columns if n in ncols]]
y =df_train['target']

n_splits = 24

models =[]
f_importance=[]
random_states=[42, 1976, 888893, 524]

new_params = [
    {'num_leaves': 337, 'max_depth': 5, 'min_data_in_leaf': 25, 
 'bagging_fraction': 0.25669443049008234,
 'feature_fraction': 0.9828506707021959, 
 'lambda_l1': 4.560681832518118, 'lambda_l2': 6.551040117920411},
    
    {'num_leaves': 368, 'max_depth': 7, 'min_data_in_leaf': 86, 
 'bagging_fraction': 0.8214445330138478, 'feature_fraction': 0.6898653647476735, 
 'lambda_l1': 1.186553228787509, 'lambda_l2': 5.543011768727234, 
 'max_bin': 485, 'bagging_freq': 56},
    
    {'num_leaves': 33, 'max_depth': 8, 'min_data_in_leaf': 88,
               'bagging_fraction': 0.7332102735163316, 'feature_fraction': 0.8186377930627372,
               'lambda_l1': 6.126340153869034, 'lambda_l2': 9.609672774031658, 
               'max_bin': 412, 'boosting': 'gbdt'},
    
        {'num_leaves': 337, 'max_depth': 5, 'min_data_in_leaf': 25, 
 'bagging_fraction': 0.25669443049008234,
 'feature_fraction': 0.9828506707021959, 
 'lambda_l1': 4.560681832518118, 'lambda_l2': 6.551040117920411},

             ]

    
for index_fold, random_state in enumerate(random_states):
    
    params.update(new_params[index_fold])
    print(params)

    cv = GroupKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kf = cv.split(X_copy, y, groups)
    for fold, (trn_idx, val_idx) in enumerate(kf):

        #print("Fold :", fold+1)

        # create dataset
        X_train, y_train = X_copy.loc[trn_idx], y[trn_idx]
        X_valid, y_valid = X_copy.loc[val_idx], y[val_idx]

        #RMSPE weight
        weights = 1/np.square(y_train)
        lgbm_train = lgbm.Dataset(X_train,y_train,weight = weights)

        weights = 1/np.square(y_valid)
        lgbm_valid = lgbm.Dataset(X_valid,y_valid,reference = lgbm_train,weight = weights)

        # model 
        model = lgbm.train(params=params,
                          train_set=lgbm_train,
                          valid_sets=[lgbm_train, lgbm_valid],
                          num_boost_round=50000,         
                          feval=feval_RMSPE,
                          verbose_eval=500,
                          categorical_feature = ['stock_id']                
                         )

        # validation 
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

        RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),5)
        print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

        #keep scores and models
        scores += RMSPE / n_splits
        models.append(model)
        #print("*" * 100)

        model.save_model('lgb_a1_model_{seed}_{fold}.txt'.format(seed=random_state,fold=fold)
                         , num_iteration=model.best_iteration)
    
print('final_score',scores/len(random_states))

In [ ]:
target = np.zeros(len(df_test))
#light gbm models
for model in models:
    pred = model.predict(df_test[list(X_train.columns)])
    target += pred / len(models)
    
y_pred = df_test[['row_id']]
y_pred = y_pred.assign(target =target)
y_pred.to_csv('submission.csv',index = False)